In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from optimus import Optimus
from optimus.profiler.functions import print_json as j

In [3]:
# Create optimus
op = Optimus(master="local", app_name= "optimus")


             ____        __  _                     
            / __ \____  / /_(_)___ ___  __  _______
           / / / / __ \/ __/ / __ `__ \/ / / / ___/
          / /_/ / /_/ / /_/ / / / / / / /_/ (__  ) 
          \____/ .___/\__/_/_/ /_/ /_/\__,_/____/  
              /_/                                  
              
Just checking that all necessary environments vars are present...
-----
PYSPARK_PYTHON=python
SPARK_HOME=C:\opt\spark\spark-2.3.1-bin-hadoop2.7
JAVA_HOME=C:\java8
-----
Starting or getting SparkSession and SparkContext...
Setting checkpoint folder ( local ). If you are in a cluster initialize optimus with master='your_ip' as param
Deleting previous folder if exists...
Creating the checkpoint directory...
Optimus successfully imported. Have fun :).


In [4]:
df = op.read.csv("Meteorite_Landings.csv", header=True)

In [5]:
df.show(10)

+-------------------+---+--------+-----------+--------+----+--------------------+----------+-----------+--------------------+
|               name| id|nametype|   recclass|mass (g)|fall|                year|    reclat|    reclong|         GeoLocation|
+-------------------+---+--------+-----------+--------+----+--------------------+----------+-----------+--------------------+
|             Aachen|  1|   Valid|         L5|      21|Fell|01/01/1880 12:00:...| 50.775000|   6.083330|(50.775000, 6.083...|
|             Aarhus|  2|   Valid|         H6|     720|Fell|01/01/1951 12:00:...| 56.183330|  10.233330|(56.183330, 10.23...|
|               Abee|  6|   Valid|        EH4|  107000|Fell|01/01/1952 12:00:...| 54.216670|-113.000000|(54.216670, -113....|
|           Acapulco| 10|   Valid|Acapulcoite|    1914|Fell|01/01/1976 12:00:...| 16.883330| -99.900000|(16.883330, -99.9...|
|            Achiras|370|   Valid|         L6|     780|Fell|01/01/1902 12:00:...|-33.166670| -64.950000|(-33.166670, -

In [6]:
df =df.cols().cast([("id","integer")])
df.dtypes

from optimus.profiler.profiler import Profiler
profiler = Profiler()

In [7]:
df.dtypes

[('name', 'string'),
 ('id', 'int'),
 ('nametype', 'string'),
 ('recclass', 'string'),
 ('mass (g)', 'string'),
 ('fall', 'string'),
 ('year', 'string'),
 ('reclat', 'string'),
 ('reclong', 'string'),
 ('GeoLocation', 'string')]

In [8]:
df =df.cols().cast([('reclat','integer')])
df.hist(['reclat','id'])

{'id': [{'bucket': 0.0, 'count': 4532, 'max': 5689, 'min': 1},
  {'bucket': 1.0, 'count': 4582, 'max': 10308, 'min': 5690},
  {'bucket': 2.0, 'count': 4591, 'max': 14976, 'min': 10309},
  {'bucket': 3.0, 'count': 4569, 'max': 19611, 'min': 14977},
  {'bucket': 4.0, 'count': 4568, 'max': 24245, 'min': 19612},
  {'bucket': 5.0, 'count': 4557, 'max': 29601, 'min': 24246},
  {'bucket': 6.0, 'count': 4560, 'max': 36353, 'min': 29602},
  {'bucket': 7.0, 'count': 4579, 'max': 46668, 'min': 36354},
  {'bucket': 8.0, 'count': 4567, 'max': 51942, 'min': 46669},
  {'bucket': 9.0, 'count': 4611, 'max': 57458, 'min': 51943}],
 'reclat': [{'bucket': None, 'count': 0, 'max': None, 'min': None},
  {'bucket': 0.0, 'count': 1423, 'max': -85, 'min': -87},
  {'bucket': 1.0, 'count': 5755, 'max': -80, 'min': -84},
  {'bucket': 2.0, 'count': 1201, 'max': -77, 'min': -79},
  {'bucket': 3.0, 'count': 4132, 'max': -73, 'min': -76},
  {'bucket': 4.0, 'count': 4699, 'max': -72, 'min': -72},
  {'bucket': 5.0, 'co

In [9]:
p =Profiler.columns(df, ['recclass','id'])

<class 'dict'>


In [10]:
j(p)

{
  "columns": {
    "recclass": {
      "uniques_count": 466,
      "p_uniques": 1.0193367748709423,
      "missing_count": 0,
      "p_missing": 0.0,
      "frequency": [
        {
          "value": "L6",
          "count": 8285,
          "percentage": 18.12275789657888
        },
        {
          "value": "H5",
          "count": 7142,
          "percentage": 15.622539154781695
        },
        {
          "value": "L5",
          "count": 4796,
          "percentage": 10.490856592877767
        },
        {
          "value": "H6",
          "count": 4528,
          "percentage": 9.904628576428385
        },
        {
          "value": "H4",
          "count": 4211,
          "percentage": 9.211217079359525
        },
        {
          "value": "LL5",
          "count": 2766,
          "percentage": 6.050398110070873
        },
        {
          "value": "LL6",
          "count": 2043,
          "percentage": 4.468894916440633
        },
        {
          "value": "L4

In [11]:
Profiler.dataset_info(df)

{'cols_count': 10,
 'missing': {'GeoLocation': 7315,
  'fall': 0,
  'id': 0,
  'mass (g)': 131,
  'name': 0,
  'nametype': 0,
  'recclass': 0,
  'reclat': 7315,
  'reclong': 7315,
  'year': 288},
 'rows_count': 45716}

In [12]:
Profiler.count_data_types(df,'*')

{'GeoLocation': {'details': {'boolean': 0,
   'float': 0,
   'integer': 0,
   'missing': 0,
   'null': 7315,
   'string': 38401},
  'type': 'string'},
 'fall': {'details': {'boolean': 0,
   'float': 0,
   'integer': 0,
   'missing': 0,
   'null': 0,
   'string': 45716},
  'type': 'string'},
 'id': {'details': {'boolean': 0,
   'float': 0,
   'integer': 45716,
   'missing': 0,
   'null': 0,
   'string': 0},
  'type': 'integer'},
 'mass (g)': {'details': {'boolean': 0,
   'float': 32526,
   'integer': 13059,
   'missing': 0,
   'null': 131,
   'string': 0},
  'type': 'float'},
 'name': {'details': {'boolean': 0,
   'float': 0,
   'integer': 0,
   'missing': 0,
   'null': 0,
   'string': 45716},
  'type': 'string'},
 'nametype': {'details': {'boolean': 0,
   'float': 0,
   'integer': 0,
   'missing': 0,
   'null': 0,
   'string': 45716},
  'type': 'string'},
 'recclass': {'details': {'boolean': 0,
   'float': 0,
   'integer': 0,
   'missing': 0,
   'null': 0,
   'string': 45716},
  'type'

## Server

In [31]:
df.cols().count_uniques('reclat')

{'reclat': 138}